In [1]:
import wrds, os
import numpy as np
import pandas as pd

In [2]:
os.chdir('..')

---

In [3]:
db = wrds.Connection()

Enter your WRDS username [maltemax]:mmx900
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: 
Create .pgpass file now [y/n]?: n
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [4]:
df = db.get_table(library = 'crsp', table = 'comphist')

In [5]:
df = df[['gvkey', 'hchgdt', 'hchgenddt', 'hfic', 'hsic', 'hcik']]

In [8]:
df['date_from'], df['date_to'] = pd.to_datetime(df['hchgdt']), pd.to_datetime(df['hchgenddt'])

df['fyear_from'] = np.where(df['date_from'].dt.month < 6,
                            df['date_from'].dt.year,
                            df['date_from'].dt.year + 1)

df['fyear_to'] = np.where(df['date_to'].dt.month < 6,
                          df['date_to'].dt.year - 1,
                          df['date_to'].dt.year)

In [9]:
df = df.loc[~(df['fyear_from'] > df['fyear_to'])].reset_index(drop = True)

In [10]:
df['fyear_to'] = df['fyear_to'].fillna(2024)

In [11]:
df = df.drop(columns = ['hchgdt', 'hchgenddt', 'date_from', 'date_to'])

In [12]:
between = df.to_dict(orient = 'index')

In [13]:
in_between = []

for k, v in between.items():
    in_between.append([i for i in range(int(v['fyear_from']), int(v['fyear_to']) + 1)])

In [14]:
df['fyear'] = pd.Series(in_between)

In [15]:
df = df.explode('fyear')

In [16]:
df = df.drop(columns = ['fyear_from', 'fyear_to'])

In [18]:
df.to_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'comphist_prepared.tsv'),
          sep = '\t', index = False)